In [17]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re , os
from collections import Counter
from tqdm import tqdm

from lyrics_preprocessing_en import lyrics_to_corpus , sentence_preprocessing
tqdm.pandas()
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Trainformers 에 BERT-base model을 불러옵니다
이미 다운로드 되어있음 ( 다운로드 안되어있을 경우 다운로드 실행 )

In [3]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [4]:
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


### 저장된 모델 불러오기
BERT 모델 학습결과 중 가장 좋은 모델 저장 경로를 불러와 현 모델에 weight 값 저장<br>
학습했던 모델과 똑같은 구조를 가진 모델을 설계하고 가중치를 입혀주어야함.

모델링 코드는 Bert_modeling 디렉터리에 있습니다.

In [5]:
checkpoint_path = os.path.join("./", 'bert_save_weight/', 'multi_emotion_weights')

In [6]:
# 학습모델과 똑같이 설계함
max_len = 70
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids,attention_mask = input_mask)[0]


embeddings = bert(input_ids,attention_mask = input_mask)[0] #(0 is the last hidden states,1 means pooler_output)
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)

y = Dense(6,activation = 'sigmoid')(out)
    
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [7]:
# 설계했던 모델에 가중치 로드
model.load_weights(checkpoint_path)

In [8]:
# 감정 에측함수
def emotion_predict(sentence):
    x_val = tokenizer(
        text=sentence,
        add_special_tokens=True,
        max_length=70,
        truncation=True,
        padding='max_length', 
        return_tensors='tf',
        return_token_type_ids = False,
        return_attention_mask = True,
        verbose = True)
    validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
    return np.argmax(validation, axis = 1)

### 데이터 불러오기

In [52]:
import pymysql
#indj 데이터 베이스 config
RDS_config = {
    # 접속할 db 정보 입력
}

# data base connect
indj_db = pymysql.connect(**RDS_config, cursorclass=pymysql.cursors.DictCursor)

In [10]:
music_analysis_query = """SELECT DISTINCT rma.music_idx, rmm.title, rmm.lyrics
FROM RENEW_music_analytics AS rma
LEFT JOIN RENEW_meta_music AS rmm ON rma.music_idx = rmm.music_idx"""

RENEW_meta_music_query = """SELECT DISTINCT rmm.music_idx , rmm.title , rmm.lyrics
FROM RENEW_meta_music AS rmm
WHERE rmm.lyrics <> 'None'"""

In [11]:
# 커서 객체 생성
cursor = indj_db.cursor()

# SQL 테이블을 pandas Dataframe 으로 불러오기
sql = RENEW_meta_music_query
cursor.execute(sql)
res = cursor.fetchall()
lyrics_df = pd.DataFrame(res)

In [13]:
#한글 영어 구분 : 한글 비중(%) 이상이면 한글 분류
def isKorean_percent(input_s , percentage):
    k_count = 0
    e_count = 0
    for c in input_s:
        if ord('가') <= ord(c) <= ord('힣'):
            k_count+=1
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            e_count+=1
    if k_count ==0:
        return 0
    else :
        percent = k_count / (k_count + e_count)
        return 1 if percent>percentage else 0

# 타이틀에 붙어있는 MR , Inst. 필터
def isRemoveData(title):
    if 'inst.' in title or 'Inst.' in title:
        return 1
    if 'Instrumental' in title or 'instrumental' in title:
        return 1
    elif 'MR' in title:
        return 1
    else :
        return 0

In [14]:
# 외국어 & 한국어 구분
lyrics_df['isKorean'] = lyrics_df['lyrics'].progress_map(lambda x : isKorean_percent(x,0.1))

In [15]:
# 외국어 필터링
en_lyrics_df = lyrics_df[lyrics_df['isKorean'] == 0]

In [18]:
# 가사 길이 구하기
en_lyrics_df['lyrics_len'] = lyrics_df['lyrics'].progress_map(lambda x : len(x))

100%|█████████████████████████████████████████████████████████████████████| 964579/964579 [00:00<00:00, 1372072.73it/s]


In [26]:
# 100자 이하 가사길이 제거
# ex) 기타 반주곡 입니다. , 가사를 넣어주세요 등등
en_lyrics_df = en_lyrics_df[en_lyrics_df['lyrics_len'] > 100]

In [27]:
from langdetect import detect #언어 판별 패키지

In [28]:
# 이 과정에서 "?????"" 만 들어있는 문장 "숫자"만 있는 문장들이 제외됨.
def language_detect(lyrics):
    """가사의 언어를 판별해주는 함수 언어를 판별할 수 없으면 no lan 반환"""
    try:
        detect_res = detect(lyrics)
        return str(detect_res)
    except:
        return "no lan"

In [29]:
# 언어 판별
en_lyrics_df['lan'] = en_lyrics_df['lyrics'].progress_map(lambda x : language_detect(x))

100%|█████████████████████████████████████████████████████████████████████████| 443402/443402 [27:07<00:00, 272.47it/s]


In [38]:
# 영어를 제외한 나머지 언어 제거
en_lyrics_df = en_lyrics_df[en_lyrics_df['lan'] == 'en']

In [45]:
# inst , MR 여부 판단
en_lyrics_df['isInst'] = en_lyrics_df['title'].progress_map(lambda x : isRemoveData(x))
# inst , MR 제거
en_lyrics_df = en_lyrics_df[en_lyrics_df['isInst'] == 0]

100%|█████████████████████████████████████████████████████████████████████| 394514/394514 [00:00<00:00, 1368023.23it/s]


In [48]:
def multi_emotion_export_persent(lyrics):
    # 가사 전처리 진행 -> 문장 중복제거 , 특수문자 제거 , 앞뒤 공백제거 , 문장 재구성
    corpus = sentence_preprocessing(lyrics_to_corpus(lyrics))
    emotion_count_list = [] 
    for i in corpus:
        # 나눠진 문장 하나씩 예측을 진행함.
        emotion_count_list.append(emotion_predict(i)[0])
    
    # 예측 결과를 합산
    emotion_count = dict(Counter(emotion_count_list).most_common(6))
    
    # 감정명 치환 { 놀람 0 , 사랑 1 , 즐거움 2 , 슬픔 3 , 분노 4 , 두려움 5 }
    emotion_rename = { 0 : '5369' , 1 : '5359' , 2 : '5370', 3 : '5364' , 4: '5365' , 5: '5368'}
    res = dict((emotion_rename[key], value) for (key, value) in emotion_count.items())
    
    # 예측결과를 비율(%)로 변경.
    total = sum(res.values())
    for emotion in list(res.keys()):
        res[emotion] = round((res[emotion] / total),5)
    
    top_dict = {}
    # 값의 크기의 따른 감정 순서 정렬( 상위 3개의 감정만 따로 뽑아주기 위함.)
    sort_dict = dict(sorted(res.items() , key=(lambda x : x[1]),reverse = True))
    sort_keys = list(sort_dict.keys())
    
       
    # 감정의 결과가 0 개 일경우 -> 0 값으로 대체함.
    if len(sort_dict) == 0:
        top_dict = {'emotion1' : 0 , 'emotion2' : 0 , 'emotion3' : 0}
    # 감정의 결과가 1개 일경우 -> 1개를 제외한 나머지를 0값으로 대체함.
    elif len(sort_dict) == 1:
        top_dict = {'emotion1' : sort_keys[0],'emotion2' : 0 , 'emotion3' : 0}
    # 감정의 결과가 2개 일경우 -> 2개를 제외한 나머지를 0값으로 대체함.
    elif len(sort_dict) == 2:
        top_dict = {'emotion1' : sort_keys[0], 'emotion2' : sort_keys[1], 'emotion3' : 0}
    # 감정의 결과가 3개 이상일 경우
    else :
        top_dict = {'emotion1' : sort_keys[0], 'emotion2' : sort_keys[1], 'emotion3' : sort_keys[2]}
    
    # 영어에는 없는 감정 "0" 값으로 추가 한국어영어 통합하기 위함
    sort_dict['5361'] = 0 # 열정
    sort_dict['5363'] = 0 # 행복
    sort_dict['5366'] = 0 # 외로움
    sort_dict['5367'] = 0 # 그리움
    
    sort_dict.update(top_dict)
    
    return sort_dict

### 미리 뽑혀진 데이터 제외

In [53]:
# 커서 객체 생성
cursor = indj_db.cursor()

# SQL 테이블을 pandas Dataframe 으로 불러오기
sql = """SELECT * FROM `indj_service`.`RENEW_lyrics_emotion_percent`;"""
cursor.execute(sql)
res = cursor.fetchall()
analysis_df = pd.DataFrame(res)

In [55]:
# 전체 데이터 - 삽입되어진 데이터 index (차집합)
unique_list = list(set(en_lyrics_df['music_idx'].to_list()) -  set(analysis_df['music_idx'].to_list()))

In [58]:
# music_idx 로 index를 바꿔주고 loc 으로 이미 추출해야만 하는 데이터 필터링(삽입되어진 데이터 제외)
en_lyrics_df = en_lyrics_df.set_index('music_idx')
en_lyrics_df = en_lyrics_df.loc[unique_list]
en_lyrics_df.reset_index(inplace=True)

### 추출

In [68]:
# 추후 DataFrame 변환을 위해 dict 형태로 추출시작
emotion_result_dict = {}

In [ ]:
for i in tqdm(range(len(en_lyrics_df))[20000:70000]):
    low = en_lyrics_df.iloc[i]
    emotion_result = multi_emotion_export_persent(low['lyrics'])
    try:
         # musict_idx 가 key 값 , emotion_result 가 value 값
        emotion_result_dict[low['music_idx']] = emotion_result
    except IndexError:
        print("{}번째 idx 에러 - 가사 : {}".format(i,low['lyrics']))

 78%|███████████████████████████████████████████████████████                | 38807/50000 [35:44:41<6:58:44,  2.24s/it]

In [72]:
emotion_result_df = pd.DataFrame(emotion_result_dict).T

In [73]:
emotion_result_df

,5370,5364,5368,5365,5369,5359,5361,5363,5366,5367,emotion1,emotion2,emotion3
31457282,0.52632,0.18421,0.13158,0.07895,0.05263,0.02632,0,0,0,0,5370,5364,5368
31457285,0.08333,0.33333,0.25,0.08333,0.25,NaN,0,0,0,0,5364,5369,5368
32505882,0.33333,0.25926,0.14815,0.14815,0.11111,NaN,0,0,0,0,5370,5364,5365
32505883,0.10417,0.5,0.14583,0.125,0.10417,0.02083,0,0,0,0,5364,5368,5365
32505884,0.10811,0.18919,0.2973,0.27027,0.10811,0.02703,0,0,0,0,5368,5365,5364
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120744,0.32143,0.10714,0.21429,0.17857,0.17857,NaN,0,0,0,0,5370,5368,5369
1120745,0.09091,0.63636,0.04545,0.09091,0.04545,0.09091,0,0,0,0,5364,5370,5359
1120746,0.30769,0.03846,0.34615,0.11538,0.15385,0.03846,0,0,0,0,5368,5370,5369
1120747,0.07895,0.28947,0.31579,0.15789,0.05263,0.10526,0,0,0,0,5368,5364,5365


In [74]:
#NaN -> 0
emotion_result_df = emotion_result_df.fillna(0)

In [76]:
# 데이터 파일로 저장
emotion_result_df.to_csv('en_emotion_data/en_210702.csv')

### 데이터 삽입

In [91]:
indj_db = pymysql.connect(**RDS_config, cursorclass=pymysql.cursors.DictCursor)
cursor = indj_db.cursor()
top_emotion_data = []
emotion_percent_data = []

for i in tqdm(range(len(emotion_result_df))):
    low = emotion_result_df.iloc[i]
    top_emotion_values = [
        low['emotion1'],
        low['emotion2'],
        low['emotion3'],
        low.name # low index
    ]
    emotion_percent_values = [
        low.name,
        low['5359'], # 사랑
        low['5370'], # 즐거움
        low['5361'], # 열정
        low['5363'], # 행복
        low['5364'], # 슬픔
        low['5365'], # 분노
        low['5366'], # 외로움
        low['5367'], # 그리움
        low['5368'], # 두려움
        low['5369']  # 놀람
    ]
    
    
    
    top_emotion_data.append(top_emotion_values)
    emotion_percent_data.append(emotion_percent_values)
    
    # 1000개의 데이터가 모일때 commit
    if i % 1000 == 0:
        sql_query = """UPDATE RENEW_music_analytics 
        SET lyrics_emotion1 = %s ,lyrics_emotion2 = %s , lyrics_emotion3 = %s
        WHERE music_idx = %s"""
        cursor.executemany(sql_query,top_emotion_data)
        indj_db.commit()
        top_emotion_data = []
        
        sql_query2 = """INSERT INTO RENEW_lyrics_emotion_percent
        (music_idx , lov , joy , pas , hap , sad ,ang , lon , mis , fea , sup)
        VALUES (%s , %s , %s , %s , %s , %s , %s, %s , %s, %s, %s)"""
        cursor.executemany(sql_query2,emotion_percent_data)
        indj_db.commit()
        emotion_percent_data = []
    
# 나머지 데이터 수정 ( n 의 배수가 아닌 데이터)
sql_query = """UPDATE RENEW_music_analytics 
    SET lyrics_emotion1 = %s ,lyrics_emotion2 = %s , lyrics_emotion3 = %s
    WHERE music_idx = %s"""
cursor.executemany(sql_query,top_emotion_data)
indj_db.commit()

sql_query2 = """INSERT INTO RENEW_lyrics_emotion_percent
    (music_idx , lov , joy , pas , hap , sad ,ang , lon , mis , fea , sup)
    VALUES (%s , %s , %s , %s , %s , %s , %s, %s , %s, %s, %s)"""
cursor.executemany(sql_query2,emotion_percent_data)
indj_db.commit()

#연결 해제
indj_db.close()
    

100%|██████████████████████████████████████████████████████████████████████████████| 9595/9595 [01:54<00:00, 83.45it/s]
